El enunciado nos da las sigueintes pistas:
- Sabemos que fue un asesinato 
- Ocurrio el 15 de enero de 2018
- En fue en la ciudad SQL City

In [2]:
import pandas as pd
import sqlite3

In [4]:
#Me conecto a la base de datos Chinook.db que tenemos la ruta 
connection=sqlite3.connect("./data/sql-murder-mystery.db")
#me creo un cursor 
cursor_TC_Murder=connection.cursor()

In [5]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas como nos dice Jaime en los videos 
def sql_query(query):

    # Ejecuta la query
    cursor_TC_Murder.execute(query) # Recuerda que sólo funcionará si has llamado cursor_bootcamp
                                    # a tu cursor, si no, cambia el nombre en todo el código de la función

    # Almacena los datos de la query(fetchall() devuelve todas las filas de la consulta en una lista de tuplas)
    ans = cursor_TC_Murder.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor_TC_Murder.description]

    return pd.DataFrame(ans,columns=names)

In [7]:
query= '''SELECT *
FROM crime_scene_report
WHERE date="20180115" AND city="SQL City"
'''
sql_query(query)

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [12]:
query= '''SELECT *
FROM person
WHERE address_street_name="Franklin Ave" AND name LIKE "%Annabel%" '''

sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [14]:
query='''SELECT id, name, address_number, address_street_name
FROM person
WHERE address_street_name="Northwestern Dr" 
ORDER BY address_number DESC LIMIT 1'''
sql_query(query)

,id,name,address_number,address_street_name
0,14887,Morty Schapiro,4919,Northwestern Dr


In [15]:
query='''SELECT  person_id, transcript
FROM interview
WHERE person_id IN (14887, 16371)'''

sql_query(query)

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [16]:
query='''SELECT*
FROM get_fit_now_member
WHERE id LIKE "48Z%"'''

sql_query(query)

,id,person_id,name,membership_start_date,membership_status
0,48Z38,49550,Tomas Baisley,20170203,silver
1,48Z7A,28819,Joe Germuska,20160305,gold
2,48Z55,67318,Jeremy Bowers,20160101,gold


In [17]:
query='''SELECT*
FROM get_fit_now_check_in
WHERE membership_id IN ("48Z7A", "48Z55")AND check_in_date=20180109'''

sql_query(query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [18]:
query='''SELECT*
FROM drivers_license
WHERE plate_number LIKE "%H42W%"
'''

sql_query(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [19]:
query='''SELECT *
FROM person
WHERE license_id IN (183779,423327,664760)
'''

sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671
